In [1]:
import findspark
#findspark.init('/home/pascalfares/DataMiningSpark/sparkhome/spark-3.0.1-bin-hadoop2.7')
findspark.init('/opt/spark/spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("QuickStart").getOrCreate()
sc = spark.sparkContext
print(sc)

<SparkContext master=local[*] appName=QuickStart>


In [ ]:
print(spark)

In [ ]:
!dir

In [ ]:
raw_data = sc.textFile("Admission_n_2020.csv")

In [ ]:
raw_data.take(5)

In [29]:
def countByCentre(raw_data, dosample=False):
    splited_row = raw_data.map(lambda line: line.split(",")
    keyCentre = splited_row.map(lambda rl: (rl[1], 1))
    reducekeys = keyCentre.reduceByKey(lambda a,b:a +b)
    return reducekeys

In [ ]:
splited_row = raw_data.map(lambda line: line.split(","))

In [ ]:
splited_row.take(5)

In [ ]:
keyCentre = splited_row.map(lambda rl: (rl[1], 1))

In [ ]:
reducekeys = keyCentre.reduceByKey(lambda a,b:a +b)

In [ ]:
reducekeys.collect()

In [30]:
countByCentre(raw_data).collect()

[('1', 164),
 ('4', 53),
 ('8', 174),
 ('IdCentre', 1),
 ('5', 61),
 ('7', 85),
 ('6', 38),
 ('3', 10)]

In [ ]:
rdc = raw_data.count()

In [ ]:
rdc

In [33]:
avgByCentre = reducekeys.map(lambda c: (c[0], c[1]/rdc))

In [34]:
avgByCentre.collect()

[('1', 0.27986348122866894),
 ('4', 0.09044368600682594),
 ('8', 0.29692832764505117),
 ('IdCentre', 0.0017064846416382253),
 ('5', 0.10409556313993173),
 ('7', 0.14505119453924914),
 ('6', 0.06484641638225255),
 ('3', 0.017064846416382253)]

In [59]:
sample_raw_data = raw_data.sample(False,0.1,88)

In [60]:
f"raw_data_count = {raw_data.count()} sample_raw_data_count = {sample_raw_data.count()}"

'raw_data_count = 586 sample_raw_data_count = 45'

In [61]:
srdc=sample_raw_data.count()

In [62]:
sampleAvgByCentre = countByCentre(sample_raw_data).map(lambda c: (c[0], c[1]/srdc))

In [63]:
sampleAvgByCentre.collect()

[('1', 0.26666666666666666),
 ('4', 0.15555555555555556),
 ('8', 0.2),
 ('7', 0.17777777777777778),
 ('3', 0.022222222222222223),
 ('5', 0.13333333333333333),
 ('6', 0.044444444444444446)]

In [48]:
keysSample = keyCentre.groupByKey().map(lambda k: {k[0]: 0.1}).reduce(lambda d1,d2: {**d1,**d2})

In [50]:
keysSample

{'1': 0.1,
 '4': 0.1,
 '8': 0.1,
 'IdCentre': 0.1,
 '5': 0.1,
 '7': 0.1,
 '6': 0.1,
 '3': 0.1}

In [65]:
sample_key_raw_data = keyCentre.sampleByKey(False,keysSample,88)

In [66]:
reducekeys = sample_key_raw_data.reduceByKey(lambda a,b:a +b)

In [67]:
reducekeys.collect()

[('1', 12), ('4', 7), ('8', 9), ('7', 8), ('3', 1), ('5', 6), ('6', 2)]

In [64]:
skrdc=sample_key_raw_data.count()
print(skrdc)
reducekeys.map(lambda c: (c[0], c[1]/skrdc)).collect()

45


[('1', 0.26666666666666666),
 ('4', 0.15555555555555556),
 ('8', 0.2),
 ('7', 0.17777777777777778),
 ('3', 0.022222222222222223),
 ('5', 0.13333333333333333),
 ('6', 0.044444444444444446)]